In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio

import os
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
%%capture
!pip install transformers
!pip install datasets

In [ ]:
%%capture
!sudo apt-get install git-lfs


In [ ]:
%%capture
!git lfs install
!--system
!--skip-repo

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install datasets>=2.0.0
!pip install evaluate
from datasets import load_dataset
from evaluate import load

data_files = {
    'train': '/content/drive/My Drive/type3/data/train_dm.csv',
    'valid': '/content/drive/My Drive/type3/data/valid_dm.csv'
    }

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_data = dataset['train']
valid_data = dataset['valid']

print(train_data)
print(valid_data)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['file', 'label', 'path'],
    num_rows: 268
})
Dataset({
    features: ['file', 'label', 'path'],
    num_rows: 56
})


In [ ]:
repo_name = "wav2vec2-large-r-300m-dm32_dementia_0810"

In [ ]:
input_col = 'path'
output_col = 'label'
audio_len = 32

In [ ]:
label_list = train_data.unique(output_col)
label_list.sort()
num_classes = len(label_list)
print(f"Number of classes: {num_classes}")
print(f"Classes: {label_list}")

Number of classes: 2
Classes: ['dementia', 'nodementia']


In [ ]:
from random import randint

def random_subsample(wav: np.ndarray, max_length: float, sample_rate: int = 16000):
    """Randomly sample chunks of `max_length` seconds from the input audio"""
    sample_length = int(round(sample_rate * max_length))
    if len(wav) <= sample_length:
        return wav
    random_offset = randint(0, len(wav) - sample_length - 1)
    return wav[random_offset : random_offset + sample_length]

In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [ ]:
model_name = "facebook/wav2vec2-xls-r-300m"
pooling_mode = "mean"

In [ ]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_classes,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
    )

setattr(config, "pooling_mode", pooling_mode)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name,)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

The target sampling rate: 16000


In [ ]:
def speech_to_array(path):
    speech, sr = torchaudio.load(path)
    transform = torchaudio.transforms.Resample(sr, 16000)
    speech = transform(speech)[0].numpy().squeeze()
    return random_subsample(speech, max_length=audio_len)


def label_to_id(label, label_list):
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label

def preprocess_fn(examples):
    speech_list = [speech_to_array(path) for path in examples[input_col]]
    target_list = [label_to_id(label, label_list) for label in examples[output_col]]
    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)
    result['labels'] = list(target_list)

    return result


In [ ]:
train_data = train_data.map(preprocess_fn, batch_size=8, batched=True, num_proc=4,)
valid_data = valid_data.map(preprocess_fn, batch_size=8, batched=True, num_proc=4,)

Map (num_proc=4):   0%|          | 0/268 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/56 [00:00<?, ? examples/s]

In [ ]:
train_data[0].keys()

dict_keys(['file', 'label', 'path', 'input_values', 'attention_mask', 'labels'])

In [ ]:
seq_len = [train_data[i]['path'] for i in range(len(train_data)) if len(train_data[i]['input_values']) < 128000]

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput

@dataclass
class SpeechClassifierModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [ ]:
import torch
import torch.nn as nn
from transformers.models.wav2vec2.modeling_wav2vec2 import (Wav2Vec2PreTrainedModel, Wav2Vec2Model)

class Wav2Vec2ClassificationHead(nn.Module):
    """head for wav2vec classification task"""
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dense(x)
        x = self.dropout(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merge_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            return torch.mean(hidden_states, dim=1)
        elif mode == "max":
            return torch.max(hidden_states, dim=1)[0]
        elif mode == "sum":
            return torch.sum(hidden_states, dim=1)
        else:
            raise ValueError(f"Unknown merge strategy: {mode}")

    def forward(
        self,
        input_values,
        attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(input_values,
                            attention_mask=attention_mask,
                            output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states,
                            return_dict=return_dict)

        hidden_states = outputs[0]
        hidden_states = self.merge_strategy(hidden_states, self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierModelOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)

In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        feature_extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`):
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"],} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.feature_extractor.pad( input_features,
                                    padding=self.padding,
                                    max_length=self.max_length,
                                    pad_to_multiple_of=self.pad_to_multiple_of,
                                    return_tensors="pt",
        )

        batch['labels'] = torch.tensor(label_features, dtype=d_type)

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(feature_extractor=feature_extractor, padding=True)

In [ ]:
import numpy as np
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)

    return {'accuracy': (preds == p.label_ids).astype(np.float32).mean().item()}

In [ ]:
#from transformers import Wav2Vec2ForSequenceClassification
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name, config=config)

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= repo_name,
    group_by_length=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    gradient_checkpointing=True,  # This enables gradient checkpointing
    num_train_epochs=22,
    logging_dir=None,
    save_steps=110,
    eval_steps=34,
    logging_steps=110,
    learning_rate=1e-4,
    save_total_limit=2,
    fp16=True,
    push_to_hub=True,
)




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer
import torch
from torch import nn
from torch.amp import autocast, GradScaler
from typing import Any, Dict, Union, Optional
from transformers.trainer_utils import seed_worker
import math, logging, time

logger = logging.getLogger("CTCTrainer")

class CTCTrainer(Trainer):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print(f"FP16 enabled: {self.args.fp16}")
        if self.args.fp16:
            self.scaler = GradScaler()
        else:
            self.scaler = None
        print(f"Scaler initialized: {self.scaler is not None}")

        # Add this safeguard
        if self.args.fp16 and self.scaler is None:
            print("Warning: FP16 is enabled but scaler is None. Reinitializing scaler.")
            self.scaler = GradScaler()

    def train(self, resume_from_checkpoint: Optional[Union[str, bool]] = None, **kwargs):
        # Initialize training
        self._train_init(resume_from_checkpoint, **kwargs)

        # Get the training dataloader
        train_dataloader = self.get_train_dataloader()

        # Total number of training steps
        num_update_steps_per_epoch = len(train_dataloader) // self.args.gradient_accumulation_steps
        num_update_steps_per_epoch = max(num_update_steps_per_epoch, 1)
        max_steps = self.args.max_steps if self.args.max_steps > 0 else num_update_steps_per_epoch * self.args.num_train_epochs
        num_train_epochs = math.ceil(max_steps / num_update_steps_per_epoch)

        # Prepare optimizer and schedule (linear warmup and decay)
        self.create_optimizer_and_scheduler(num_training_steps=max_steps)

        # Train!
        logger.info("***** Running training *****")
        logger.info(f"  Num examples = {self.num_examples(train_dataloader)}")
        logger.info(f"  Num Epochs = {num_train_epochs}")
        logger.info(f"  Instantaneous batch size per device = {self.args.per_device_train_batch_size}")
        logger.info(f"  Total train batch size = {self.args.train_batch_size * self.args.gradient_accumulation_steps}")
        logger.info(f"  Gradient Accumulation steps = {self.args.gradient_accumulation_steps}")
        logger.info(f"  Total optimization steps = {max_steps}")

        self.state.epoch = 0
        start_time = time.time()
        epochs_trained = 0
        steps_trained_in_current_epoch = 0

        # Check if continuing training from a checkpoint
        if resume_from_checkpoint is not None:
            self._load_from_checkpoint(resume_from_checkpoint)
            epochs_trained = self.state.global_step // num_update_steps_per_epoch
            steps_trained_in_current_epoch = self.state.global_step % num_update_steps_per_epoch

        # Update the references
        self.callback_handler.model = self.model
        self.callback_handler.optimizer = self.optimizer
        self.callback_handler.lr_scheduler = self.lr_scheduler
        self.callback_handler.train_dataloader = train_dataloader

        # Training loop
        for epoch in range(epochs_trained, num_train_epochs):
            self.state.epoch = epoch
            self.callback_handler.on_epoch_begin(self.args, self.state, self.control)

            for step, inputs in enumerate(train_dataloader):
                # Skip past any already trained steps if resuming training
                if steps_trained_in_current_epoch > 0:
                    steps_trained_in_current_epoch -= 1
                    continue

                # Training step
                self.callback_handler.on_step_begin(self.args, self.state, self.control)

                # Perform a training step
                loss = self.training_step(self.model, inputs)

                # Check scaler state
                if self.args.fp16 and self.scaler is not None:
                    print(f"Scaler state after training_step: {self.scaler._scale}")

                # Accumulate gradients
                if self.args.gradient_accumulation_steps > 1:
                    loss = loss / self.args.gradient_accumulation_steps

                if (step + 1) % self.args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                    # Perform optimizer step
                    print(f"Performing optimizer step at global_step {self.state.global_step}")
                    if self.args.fp16 and self.scaler is not None:
                        print(f"Scaler state before optimizer_step: {self.scaler._scale}")

                    self.optimizer_step(self.optimizer, self.model, self.state.global_step)

                    if self.args.fp16 and self.scaler is not None:
                        print(f"Scaler state after optimizer_step: {self.scaler._scale}")

                self.state.global_step += 1
                self.state.epoch = epoch + (step + 1) / len(train_dataloader)
                self.control = self.callback_handler.on_step_end(self.args, self.state, self.control)

                if self.control.should_epoch_stop or self.control.should_training_stop:
                    break

            self.callback_handler.on_epoch_end(self.args, self.state, self.control)
            self.control = self.callback_handler.on_save(self.args, self.state, self.control)

            if self.control.should_training_stop:
                break

        if self.args.past_index and hasattr(self, "_past"):
            # Clean the state at the end of training
            delattr(self, "_past")

        logger.info("\n\nTraining completed. Do not forget to share your model on huggingface.co/models =)\n\n")

        return TrainOutput(self.state.global_step, self.state.training_loss, self.state.metrics)

    def _train_init(self, resume_from_checkpoint, **kwargs):
        # Initialize training parameters
        self._memory_tracker.start()
        self._hp_search_setup(None)
        self._load_from_checkpoint(resume_from_checkpoint)
        self._setup_training()

    def _load_from_checkpoint(self, resume_from_checkpoint):
        if resume_from_checkpoint is not None:
            if not os.path.isfile(os.path.join(resume_from_checkpoint, WEIGHTS_NAME)):
                raise ValueError(f"Can't find a valid checkpoint at {resume_from_checkpoint}")

            logger.info(f"Loading model from {resume_from_checkpoint}")
            if os.path.isfile(os.path.join(resume_from_checkpoint, CONFIG_NAME)):
                config = PretrainedConfig.from_json_file(os.path.join(resume_from_checkpoint, CONFIG_NAME))
                checkpoint_version = config.transformers_version
                if checkpoint_version is not None and checkpoint_version != __version__:
                    logger.warning(
                        f"You are resuming training from a checkpoint trained with {checkpoint_version} of "
                        f"Transformers but your current version is {__version__}. This is not recommended and could "
                        "yield to errors or unwanted behaviors."
                    )

            if not self.args.deepspeed:
                # We load the model state dict on the CPU to avoid an OOM error.
                state_dict = torch.load(os.path.join(resume_from_checkpoint, WEIGHTS_NAME), map_location="cpu")
                # If the model is on the GPU, it still works!
                self._load_state_dict_in_model(state_dict)
                # release memory
                del state_dict

    def _setup_training(self):
        # Activate gradient checkpointing if needed
        if self.args.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # wrap model
        self.model_wrapped = self.model

        # for the rest of this function `model` is the outside model, whether it was wrapped or not
        if self.model_wrapped is not self.model:
            self.model_wrapped = self.model

        # important: at this point:
        # self.model         is the Transformers Model
        # self.model_wrapped is DDP(Transformers Model), Deepspeed(Transformers Model), etc.

    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)

        # Use autocast for mixed precision training, specify device type explicitly
        with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu', enabled=self.args.fp16):
            loss = self.compute_loss(model, inputs)

        print(f"Raw loss: {loss.item()}")

        # Ensure that loss is valid
        if not torch.isfinite(loss):
            print(f"Warning: Non-finite loss detected: {loss}")
            return loss.detach()  # Return early to avoid further issues

        # Handle gradient scaling and backward pass
        if self.args.fp16 and self.scaler is not None:
            scaled_loss = self.scaler.scale(loss)
            print(f"Scaled loss: {scaled_loss.item()}")
            scaled_loss.backward()
            print(f"Backward pass completed")
        else:
            loss.backward()

        print(f"Scaler state after backward: {self.scaler._scale if self.scaler else 'No scaler'}")

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        return loss.detach()

    def optimizer_step(self, optimizer: torch.optim.Optimizer, model: nn.Module, step: int):
        print(f"Optimizer step - FP16: {self.args.fp16}, Scaler: {self.scaler is not None}")
        if self.args.fp16 and self.scaler is not None:
            print(f"Scaler state at start of optimizer_step: {self.scaler._scale}")

            # Check if there are any scaled gradients
            grads = [p.grad for p in model.parameters() if p.grad is not None]
            if grads:
                print(f"Number of gradient tensors: {len(grads)}")
                print(f"Grad norms: {[g.norm().item() for g in grads]}")
                try:
                    # Unscale gradients before clipping to prevent overflows
                    self.scaler.unscale_(optimizer)
                    print("Unscale successful")
                    torch.nn.utils.clip_grad_norm_(model.parameters(), self.args.max_grad_norm)

                    # Perform the optimizer step using the GradScaler
                    self.scaler.step(optimizer)
                    self.scaler.update()  # Update the scaler for the next step
                    print(f"Scaler state after update: {self.scaler._scale}")
                except RuntimeError as e:
                    print(f"RuntimeError in optimizer_step: {e}")
                    optimizer.zero_grad()
                    return
            else:
                print("No gradients to unscale, skipping optimizer step")
        else:
            # Clip gradients for fp32 training
            torch.nn.utils.clip_grad_norm_(model.parameters(), self.args.max_grad_norm)

            # Normal optimizer step
            optimizer.step()

        # Zero out the gradients for the next step
        optimizer.zero_grad()
        print("Gradients zeroed")



In [ ]:
from transformers import Trainer
import torch
from torch import nn
from torch.amp import autocast, GradScaler
from typing import Any, Dict, Union, Optional
from transformers.trainer_utils import seed_worker
import math, logging, time

logger = logging.getLogger("CTCTrainer")

class CTCTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print(f"FP16 enabled: {self.args.fp16}")
        if self.args.fp16:
            self.scaler = GradScaler()
        else:
            self.scaler = None
        print(f"Scaler initialized: {self.scaler is not None}")

        # Add this safeguard
        if self.args.fp16 and self.scaler is None:
            print("Warning: FP16 is enabled but scaler is None. Reinitializing scaler.")
            self.scaler = GradScaler()

    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)

        # Use autocast for mixed precision training, specify device type explicitly
        with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu', enabled=self.args.fp16):
            loss = self.compute_loss(model, inputs)

        print(f"Raw loss: {loss.item()}")

        # Ensure that loss is valid
        if not torch.isfinite(loss):
            print(f"Warning: Non-finite loss detected: {loss}")
            return loss.detach()  # Return early to avoid further issues

        # Handle gradient scaling and backward pass
        if self.args.fp16 and self.scaler is not None:
            scaled_loss = self.scaler.scale(loss)
            print(f"Scaled loss: {scaled_loss.item()}")
            scaled_loss.backward()
            print(f"Backward pass completed")
        else:
            loss.backward()

        print(f"Scaler state after backward: {self.scaler._scale if self.scaler else 'No scaler'}")

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        return loss.detach()

    def optimizer_step(self, optimizer: torch.optim.Optimizer, model: nn.Module, step: int):
        print(f"Optimizer step - FP16: {self.args.fp16}, Scaler: {self.scaler is not None}")
        if self.args.fp16 and self.scaler is not None:
            print(f"Scaler state at start of optimizer_step: {self.scaler._scale}")

            # Check if there are any scaled gradients
            grads = [p.grad for p in model.parameters() if p.grad is not None]
            if grads:
                print(f"Number of gradient tensors: {len(grads)}")
                print(f"Grad norms: {[g.norm().item() for g in grads]}")
                try:
                    # Unscale gradients before clipping to prevent overflows
                    self.scaler.unscale_(optimizer)
                    print("Unscale successful")
                    torch.nn.utils.clip_grad_norm_(model.parameters(), self.args.max_grad_norm)

                    # Perform the optimizer step using the GradScaler
                    self.scaler.step(optimizer)
                    self.scaler.update()  # Update the scaler for the next step
                    print(f"Scaler state after update: {self.scaler._scale}")
                except RuntimeError as e:
                    print(f"RuntimeError in optimizer_step: {e}")
                    optimizer.zero_grad()
                    return
            else:
                print("No gradients to unscale, skipping optimizer step")
        else:
            # Clip gradients for fp32 training
            torch.nn.utils.clip_grad_norm_(model.parameters(), self.args.max_grad_norm)

            # Normal optimizer step
            optimizer.step()

        # Zero out the gradients for the next step
        optimizer.zero_grad()
        print("Gradients zeroed")


In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=valid_data,
    tokenizer=feature_extractor,
)

FP16 enabled: True
Scaler initialized: True


/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Raw loss: 0.6740212440490723
Scaled loss: 0.6740212440490723


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


In [ ]:
trainer.push_to_hub(repo_name)